# <center>__**CAPSTONE PROJECT : BATTLE OF NEIGHBOURHOODS**__</center>

#### AUTHOR : PARVEEN SAHRAWAT
#### DATE : 07 DEC 2019

Contents:

>1.  Introduction.
>>    1.1. Topic.<br>
>>    1.2. Problem Statement<br>
>>    1.2. Audience<br>

>2.  Data.
>>    2.1. Data Requirement<br>
>>    2.2. Data Source<br>
    
>3.  Methodology.
>4.  Results
>5.  Discussion
>6.  Conclussion
    


Introduction

Topic: Changi Business Park (CBP)in Singapore is one of the main business hubs. It is second Business park of Singapore following the International Business Park (IBP)in Jurong East. CBP comprises a mix of high technology business, data and Software enterprises, research and development and knowledge intensive facilities. As a result lot of people from other parts of the world migrate to this area with relation to their business and jobs. One of the concern of people is to know the neighbourhood,venues nearby and place to stay. Particularly for Indians to identify the places where Indian food is easily available.

Problem Statement: To find the neighbourhood venues near to Changi Business Park and a suitable place to stay.

Audience: This report is relevant to people who are moving in CBP Singapore. Particularly it will be addressing Indians, who are coming to Singapore and likely to work in CBP area.



Data

Data Requirement: Following data will be required to analyse and reach to the conclusion:

1. List of neighbourhood.<br>
2. List of venues like Restaurants, Malls and shopping centers and other amenities.<br>
3. Distance from workplace (CBP).<br>
4. Transportation facilities.<br>



Data Source: Following Datasources will be used to get the relevant data.

1. Neighbourhood and Postal code will be compiled from https://en.wikipedia.org/wiki/Postal_codes_in_Singapore .<br>
2. Algorithm using nomantim will be used to get geodata for various venues and neighbourhood.<br>
3. Singapore MRT stations will be opbtatined from https://data.world/hxchua/train-stations-in-singapore .<br>
4. For rental rates we will get CSV from https://data.gov.sg/ which publishes district wise rental rates for various type of properties


3. Methodology section:

In this section we will describe the usage of data collected and how it is converted to give a meaningful insight towards solving the problem.

The process for reaching to a conclusion to finalise the location to rent a place to reside while working in CBP is to plot following data on the MAP to determine which place is most suitable as per below conditions.

    1. Place should not be far away from Office area in CBP (We are exploring places within 3 KM from CBP).
    2. There should be availability of Indian Food in nearby area.
    3. Apart from point 2 below venues are also preferred.
        - Breakfast Spot
        - Café
        - Coffee Shop
        - Discount Store
        - Food Court
        - Ice Cream Shop
        - Malay Restaurant
        - Shopping Mall
        - Supermarket
     4. Place should be near to MRT.
     5. Place should be affordable.

Visualising on map gives advantage that popups allow us quick identification of location, rental cost and nearby venues. 


Importing Libraries

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
!pip install geocoder
!pip install geopy
!pip install BeautifulSoup4
!pip install lxml==3.4.2
print('Libraries imported')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge


In [2]:
import requests
import json
import random
import geocoder
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from IPython.display import display_html
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import time

Getting Latitude and Longitude of Changi Business Park (Our area of focus)

In [3]:
address = 'Changi Business Park, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore home are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Singapore home are 1.3350665, 103.965228.


In [4]:
CLIENT_ID = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' 
VERSION = 'xxxxxxxx'
LIMIT = 100
RADIUS = 3000

In [5]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    RADIUS, 
    LIMIT)

In [6]:
results = requests.get(url).json()

In [7]:
def get_category_type(row):
    try:
        catagories_list = row['categories']
    except:
        catagories_list = row['venue.categories']
        
    if (len(catagories_list)==0):
        return None
    else:
        return catagories_list[0]['name']

List of venues near to Changi Business Part (With in radius of 3 KM)

In [8]:
venues = results['response']['groups'][0]['items']
NearbyVenues = json_normalize(venues)
filtered_columns = ['venue.name', 
                    'venue.categories', 
                    'venue.location.postalCode', 
                    'venue.location.neighborhood',
                    'venue.location.lat',
                    'venue.location.lng',
                    'venue.location.distance',
                    'venue.location.address'
                   ]

NearbyVenues = NearbyVenues.loc[:, filtered_columns]
NearbyVenues['venue.categories'] = NearbyVenues.apply(get_category_type, axis=1)
NearbyVenues.columns = [col.split(".")[-1] for col in NearbyVenues.columns]
NearbyVenues.head()

,name,categories,postalCode,neighborhood,lat,lng,distance,address
0,Huggs JPM @ Changi City (huggs coffee),Café,486018,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1
1,Capri By Fraser,Hotel,486037,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1
2,SONG FA Bak Kut Teh,Chinese Restaurant,486017,NaN,1.335363,103.964323,106,UE BizHub East
3,Nike Factory Store,Sporting Goods Shop,486038,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point"
4,Starbucks,Coffee Shop,486038,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point


In [9]:
def flt_to_fix(num):
    res = '{0: .4f}'.format(num)
    return res

In [10]:
Lat_Long = NearbyVenues.as_matrix(columns=NearbyVenues.columns[4:6])
cols = ['lat', 'lng', 'lat_long']
LL_LST = []
for value in (Lat_Long):
    latitude = value[0]
    longitude = value[1]
    temp_lat = flt_to_fix(latitude)
    temp_lng = flt_to_fix(longitude)
    lat_lng = (str(temp_lat)+','+str(temp_lng))
    LL_LST.append([latitude, longitude, lat_lng])
df_LL = pd.DataFrame(LL_LST, columns=cols)
df_LL.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


,lat,lng,lat_long
0,1.334441,103.964183,"1.3344, 103.9642"
1,1.333618,103.963651,"1.3336, 103.9637"
2,1.335363,103.964323,"1.3354, 103.9643"
3,1.334440,103.962490,"1.3344, 103.9625"
4,1.334180,103.962616,"1.3342, 103.9626"


In [11]:
NearbyVenues = pd.merge(NearbyVenues, df_LL, on = ['lat', 'lng'])
NearbyVenues.head()

,name,categories,postalCode,neighborhood,lat,lng,distance,address,lat_long
0,Huggs JPM @ Changi City (huggs coffee),Café,486018,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1,"1.3344, 103.9642"
1,Capri By Fraser,Hotel,486037,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1,"1.3336, 103.9637"
2,SONG FA Bak Kut Teh,Chinese Restaurant,486017,NaN,1.335363,103.964323,106,UE BizHub East,"1.3354, 103.9643"
3,Nike Factory Store,Sporting Goods Shop,486038,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point","1.3344, 103.9625"
4,Starbucks,Coffee Shop,486038,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point,"1.3342, 103.9626"


In [12]:
SG_LL_PC_DF = pd.read_csv('LL_PC_SG.csv')
SG_LL_PC_DF = SG_LL_PC_DF.rename(columns={
    'Lat' : 'lat',
    'Lng' : 'lng',
    'PostalCode' : 'PostCode'
})
SG_LL_PC_DF.head()

,lat,lng,PostCode
0,1.334441,103.964183,486018
1,1.333618,103.963651,486038
2,1.334440,103.962490,486038
3,1.334180,103.962616,486038
4,1.335085,103.964109,486017


In [13]:
Lat_Long = SG_LL_PC_DF.as_matrix(columns=SG_LL_PC_DF.columns[0:2])
cols = ['lat', 'lng', 'lat_long']
LL_LST = []
for value in (Lat_Long):
    latitude = value[0]
    longitude = value[1]
    temp_lat = flt_to_fix(latitude)
    temp_lng = flt_to_fix(longitude)
    lat_lng = (str(temp_lat)+','+str(temp_lng))
    LL_LST.append([latitude, longitude, lat_lng])
df_LL_1 = pd.DataFrame(LL_LST, columns=cols)
df_LL_1.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


,lat,lng,lat_long
0,1.334441,103.964183,"1.3344, 103.9642"
1,1.333618,103.963651,"1.3336, 103.9637"
2,1.334440,103.962490,"1.3344, 103.9625"
3,1.334180,103.962616,"1.3342, 103.9626"
4,1.335085,103.964109,"1.3351, 103.9641"


In [14]:
SG_LL_PC_DF = pd.merge(SG_LL_PC_DF, df_LL_1, on=['lat', 'lng'])
SG_LL_PC_DF=SG_LL_PC_DF.drop(['lat', 'lng'], axis=1)
SG_LL_PC_DF.head()

,PostCode,lat_long
0,486018,"1.3344, 103.9642"
1,486038,"1.3336, 103.9637"
2,486038,"1.3344, 103.9625"
3,486038,"1.3342, 103.9626"
4,486017,"1.3351, 103.9641"


In [15]:
NearbyVenues_PC = pd.merge(NearbyVenues, SG_LL_PC_DF, on='lat_long', how='left')
NearbyVenues_PC

,name,categories,postalCode,neighborhood,lat,lng,distance,address,lat_long,PostCode
0,Huggs JPM @ Changi City (huggs coffee),Café,486018,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1,"1.3344, 103.9642",486018.0
1,Capri By Fraser,Hotel,486037,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1,"1.3336, 103.9637",486038.0
2,SONG FA Bak Kut Teh,Chinese Restaurant,486017,NaN,1.335363,103.964323,106,UE BizHub East,"1.3354, 103.9643",NaN
3,Nike Factory Store,Sporting Goods Shop,486038,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point","1.3344, 103.9625",486038.0
4,Starbucks,Coffee Shop,486038,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point,"1.3342, 103.9626",486038.0
...,...,...,...,...,...,...,...,...,...,...
107,Tampines Round Market & Food Centre,Food Court,521137,NaN,1.345790,103.944525,2594,137 Tampines Street 11,"1.3458, 103.9445",521137.0
108,"201C, Tampines St 21",Coffee Shop,NaN,NaN,1.352559,103.952867,2384,"201C, Tampines St 21","1.3526, 103.9529",NaN
109,Four Leaves,Bakery,529510,NaN,1.352473,103.945128,2959,"#B1-22, Tampines Mall","1.3525, 103.9451",529510.0
110,EnergyOne Tampines,Gym,528882,NaN,1.344600,103.941256,2871,SAFRA Tampines,"1.3446, 103.9413",528882.0


In [16]:
NearbyVenues_PC.loc[NearbyVenues_PC['PostCode'].isnull(), ['PostCode']] = NearbyVenues_PC['postalCode']
NearbyVenues_PC.loc[NearbyVenues_PC['categories'] == 'South Indian Restaurant', ['categories']] = 'Indian Restaurant'
NearbyVenues_PC.head()

,name,categories,postalCode,neighborhood,lat,lng,distance,address,lat_long,PostCode
0,Huggs JPM @ Changi City (huggs coffee),Café,486018,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1,"1.3344, 103.9642",486018
1,Capri By Fraser,Hotel,486037,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1,"1.3336, 103.9637",486038
2,SONG FA Bak Kut Teh,Chinese Restaurant,486017,NaN,1.335363,103.964323,106,UE BizHub East,"1.3354, 103.9643",486017
3,Nike Factory Store,Sporting Goods Shop,486038,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point","1.3344, 103.9625",486038
4,Starbucks,Coffee Shop,486038,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point,"1.3342, 103.9626",486038


In [17]:
NearbyVenues_PC.dropna(subset=['PostCode'], how='all', inplace=True)
NearbyVenues_PC.shape

(103, 10)

In [18]:
NearbyVenues_PC=NearbyVenues_PC.drop(['postalCode'], axis=1)
NearbyVenues_PC=NearbyVenues_PC.rename(columns={'PostCode' : 'PostalCode'})
NearbyVenues_PC

,name,categories,neighborhood,lat,lng,distance,address,lat_long,PostalCode
0,Huggs JPM @ Changi City (huggs coffee),Café,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1,"1.3344, 103.9642",486018
1,Capri By Fraser,Hotel,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1,"1.3336, 103.9637",486038
2,SONG FA Bak Kut Teh,Chinese Restaurant,NaN,1.335363,103.964323,106,UE BizHub East,"1.3354, 103.9643",486017
3,Nike Factory Store,Sporting Goods Shop,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point","1.3344, 103.9625",486038
4,Starbucks,Coffee Shop,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point,"1.3342, 103.9626",486038
...,...,...,...,...,...,...,...,...,...
106,Wolf Burgers,Burger Joint,NaN,1.334125,103.962919,277,#01-43 Changi City Point,"1.3341, 103.9629",486038
107,Tampines Round Market & Food Centre,Food Court,NaN,1.345790,103.944525,2594,137 Tampines Street 11,"1.3458, 103.9445",521137
109,Four Leaves,Bakery,NaN,1.352473,103.945128,2959,"#B1-22, Tampines Mall","1.3525, 103.9451",529510
110,EnergyOne Tampines,Gym,NaN,1.344600,103.941256,2871,SAFRA Tampines,"1.3446, 103.9413",528882


In [19]:
NearbyVenues_PC['PostalCode'] = NearbyVenues_PC['PostalCode'].astype(str)
NearbyVenues_PC['PostalSector'] = NearbyVenues_PC['PostalCode'].str[:2]
NearbyVenues_PC

,name,categories,neighborhood,lat,lng,distance,address,lat_long,PostalCode,PostalSector
0,Huggs JPM @ Changi City (huggs coffee),Café,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1,"1.3344, 103.9642",486018.0,48
1,Capri By Fraser,Hotel,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1,"1.3336, 103.9637",486038.0,48
2,SONG FA Bak Kut Teh,Chinese Restaurant,NaN,1.335363,103.964323,106,UE BizHub East,"1.3354, 103.9643",486017,48
3,Nike Factory Store,Sporting Goods Shop,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point","1.3344, 103.9625",486038.0,48
4,Starbucks,Coffee Shop,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point,"1.3342, 103.9626",486038.0,48
...,...,...,...,...,...,...,...,...,...,...
106,Wolf Burgers,Burger Joint,NaN,1.334125,103.962919,277,#01-43 Changi City Point,"1.3341, 103.9629",486038.0,48
107,Tampines Round Market & Food Centre,Food Court,NaN,1.345790,103.944525,2594,137 Tampines Street 11,"1.3458, 103.9445",521137.0,52
109,Four Leaves,Bakery,NaN,1.352473,103.945128,2959,"#B1-22, Tampines Mall","1.3525, 103.9451",529510.0,52
110,EnergyOne Tampines,Gym,NaN,1.344600,103.941256,2871,SAFRA Tampines,"1.3446, 103.9413",528882.0,52


In [20]:
NearbyVenues_PC.shape

(103, 10)

In [21]:
website_url = requests.get('https://en.wikipedia.org/wiki/Postal_codes_in_Singapore').text
#BS = BeautifulSoup(website_url, 'lxml')
#print(BS.prettify())
#PostCode_table = str(BS.table)
#display_html(PostCode_table, raw=True)
SG_PostCode_DF=pd.read_html(website_url)[0]
#SG_PostCode_DF.head()

In [22]:
SG_PostCode_DF = SG_PostCode_DF.rename(columns={
    'Postal district' : 'PostalDistrict',
    'Postal sector(1st 2 digits of 6-digit postal codes)': 'PostalSector',
    'General location' : 'Neighbourhoods'
}) 
SG_PostCode_DF = SG_PostCode_DF.reset_index(drop=True)
SG_PostCode_DF.head()

,PostalDistrict,PostalSector,Neighbourhoods
0,1,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park"
1,2,"07, 08","Anson, Tanjong Pagar"
2,3,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru"
3,4,"09, 10","Telok Blangah, Harbourfront"
4,5,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."


In [23]:
SG_PostCode_DF_1=pd.DataFrame(SG_PostCode_DF['PostalSector'].str.split(',').tolist(), index=SG_PostCode_DF.Neighbourhoods).stack()
SG_PostCode_DF_1=SG_PostCode_DF_1.reset_index([0, 'Neighbourhoods'])
SG_PostCode_DF_1.columns = ['Neighbourhoods', 'PostalSector']
SG_PostCode_DF_1.reset_index()

,index,Neighbourhoods,PostalSector
0,0,"Raffles Place, Cecil, Marina, People's Park",01
1,1,"Raffles Place, Cecil, Marina, People's Park",02
2,2,"Raffles Place, Cecil, Marina, People's Park",03
3,3,"Raffles Place, Cecil, Marina, People's Park",04
4,4,"Raffles Place, Cecil, Marina, People's Park",05
...,...,...,...
76,76,"Upper Thomson, Springleaf",78
77,77,"Yishun, Sembawang",75
78,78,"Yishun, Sembawang",76
79,79,Seletar,79


In [24]:
SG_PostSect_DF = pd.merge(SG_PostCode_DF, SG_PostCode_DF_1, on='Neighbourhoods', how='right')
SG_PostSect_DF.drop(['PostalSector_x'], axis=1, inplace=True)
SG_PostSect_DF.rename(columns={'PostalSector_y' : 'PostalSector'}, inplace=True)
SG_PostSect_DF

,PostalDistrict,Neighbourhoods,PostalSector
0,1,"Raffles Place, Cecil, Marina, People's Park",01
1,1,"Raffles Place, Cecil, Marina, People's Park",02
2,1,"Raffles Place, Cecil, Marina, People's Park",03
3,1,"Raffles Place, Cecil, Marina, People's Park",04
4,1,"Raffles Place, Cecil, Marina, People's Park",05
...,...,...,...
76,26,"Upper Thomson, Springleaf",78
77,27,"Yishun, Sembawang",75
78,27,"Yishun, Sembawang",76
79,28,Seletar,79


In [25]:
SG_PostSect_DF['PostalSector'] = SG_PostSect_DF['PostalSector'].str.strip()
SG_PostSect_DF

,PostalDistrict,Neighbourhoods,PostalSector
0,1,"Raffles Place, Cecil, Marina, People's Park",01
1,1,"Raffles Place, Cecil, Marina, People's Park",02
2,1,"Raffles Place, Cecil, Marina, People's Park",03
3,1,"Raffles Place, Cecil, Marina, People's Park",04
4,1,"Raffles Place, Cecil, Marina, People's Park",05
...,...,...,...
76,26,"Upper Thomson, Springleaf",78
77,27,"Yishun, Sembawang",75
78,27,"Yishun, Sembawang",76
79,28,Seletar,79


In [26]:
SG_PostSect_DF['PostalSector'] = SG_PostSect_DF['PostalSector'].astype(str)
SG_PostSect_DF['PostalDistrict'] = SG_PostSect_DF['PostalDistrict'].astype(str)
SG_PostSect_DF.to_csv('SG_PostSect.csv')
NearbyVenues_PC['PostalSector'] = NearbyVenues_PC['PostalSector'].astype(str)

In [27]:
temp_df = SG_PostSect_DF
temp_df = temp_df[temp_df['Neighbourhoods'].isnull()]
temp_df

,PostalDistrict,Neighbourhoods,PostalSector


In [28]:
CBP_Nearby_Venues = pd.merge(NearbyVenues_PC, SG_PostSect_DF,  on='PostalSector', how='left')
CBP_Nearby_Venues

,name,categories,neighborhood,lat,lng,distance,address,lat_long,PostalCode,PostalSector,PostalDistrict,Neighbourhoods
0,Huggs JPM @ Changi City (huggs coffee),Café,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1,"1.3344, 103.9642",486018.0,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive"
1,Capri By Fraser,Hotel,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1,"1.3336, 103.9637",486038.0,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive"
2,SONG FA Bak Kut Teh,Chinese Restaurant,NaN,1.335363,103.964323,106,UE BizHub East,"1.3354, 103.9643",486017,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive"
3,Nike Factory Store,Sporting Goods Shop,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point","1.3344, 103.9625",486038.0,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive"
4,Starbucks,Coffee Shop,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point,"1.3342, 103.9626",486038.0,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive"
...,...,...,...,...,...,...,...,...,...,...,...,...
98,Wolf Burgers,Burger Joint,NaN,1.334125,103.962919,277,#01-43 Changi City Point,"1.3341, 103.9629",486038.0,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive"
99,Tampines Round Market & Food Centre,Food Court,NaN,1.345790,103.944525,2594,137 Tampines Street 11,"1.3458, 103.9445",521137.0,52,18,"Simei, Tampines, Pasir Ris"
100,Four Leaves,Bakery,NaN,1.352473,103.945128,2959,"#B1-22, Tampines Mall","1.3525, 103.9451",529510.0,52,18,"Simei, Tampines, Pasir Ris"
101,EnergyOne Tampines,Gym,NaN,1.344600,103.941256,2871,SAFRA Tampines,"1.3446, 103.9413",528882.0,52,18,"Simei, Tampines, Pasir Ris"


In [29]:
print('Map displaying venues nearby to Changi Business Park (with in range of 3 KM)')
Map_CBP=folium.Map(location=[latitude, longitude], zoom_start=14)
for lat, lng, label in zip(NearbyVenues['lat'], NearbyVenues['lng'], NearbyVenues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color = '#3186cc',
        fill_opacity=0.7,
        parse_html=True
    ).add_to(Map_CBP)
Map_CBP

Map displaying venues nearby to Changi Business Park (with in range of 3 KM)


In [30]:
CBP_Onehot = pd.get_dummies(CBP_Nearby_Venues[['categories']], prefix='', prefix_sep='')
CBP_Onehot['Neighbourhoods'] = CBP_Nearby_Venues['Neighbourhoods']
fixed_columns = [CBP_Onehot.columns[-1]]+list(CBP_Onehot.columns[:-1])
CBP_Onehot = CBP_Onehot[fixed_columns]
CBP_Onehot

,Neighbourhoods,Airport Lounge,Airport Service,Asian Restaurant,BBQ Joint,Bakery,Bar,Breakfast Spot,Burger Joint,Café,...,Pool,Resort,Sandwich Place,Seafood Restaurant,Shopping Mall,Skate Park,Soup Place,Sporting Goods Shop,Steakhouse,Supermarket
0,"Bedok, Upper East Coast, Eastwood, Kew Drive",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"Bedok, Upper East Coast, Eastwood, Kew Drive",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bedok, Upper East Coast, Eastwood, Kew Drive",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Bedok, Upper East Coast, Eastwood, Kew Drive",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,"Bedok, Upper East Coast, Eastwood, Kew Drive",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,"Bedok, Upper East Coast, Eastwood, Kew Drive",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
99,"Simei, Tampines, Pasir Ris",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,"Simei, Tampines, Pasir Ris",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,"Simei, Tampines, Pasir Ris",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
CBP_Grouped = CBP_Onehot.groupby('Neighbourhoods').mean().reset_index()
CBP_Grouped

,Neighbourhoods,Airport Lounge,Airport Service,Asian Restaurant,BBQ Joint,Bakery,Bar,Breakfast Spot,Burger Joint,Café,...,Pool,Resort,Sandwich Place,Seafood Restaurant,Shopping Mall,Skate Park,Soup Place,Sporting Goods Shop,Steakhouse,Supermarket
0,"Bedok, Upper East Coast, Eastwood, Kew Drive",0.000000,0.000000,0.028169,0.028169,0.014085,0.014085,0.014085,0.014085,0.070423,...,0.000000,0.000000,0.028169,0.042254,0.042254,0.000000,0.000000,0.028169,0.000000,0.014085
1,"Loyang, Changi",0.083333,0.166667,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.166667,...,0.000000,0.166667,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000
2,"Simei, Tampines, Pasir Ris",0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.052632,...,0.052632,0.000000,0.052632,0.000000,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000


In [32]:
CBP_Grouped.columns

Index(['Neighbourhoods', 'Airport Lounge', 'Airport Service',
       'Asian Restaurant', 'BBQ Joint', 'Bakery', 'Bar', 'Breakfast Spot',
       'Burger Joint', 'Café', 'Chinese Restaurant', 'Club House',
       'Coffee Shop', 'Coworking Space', 'Deli / Bodega', 'Dessert Shop',
       'Discount Store', 'Dumpling Restaurant', 'Electronics Store',
       'Event Space', 'Fast Food Restaurant', 'Filipino Restaurant',
       'Food Court', 'Golf Course', 'Gym', 'Harbor / Marina', 'Hotel',
       'Ice Cream Shop', 'Indian Restaurant', 'Japanese Restaurant',
       'Kitchen Supply Store', 'Malay Restaurant', 'Noodle House', 'Pool',
       'Resort', 'Sandwich Place', 'Seafood Restaurant', 'Shopping Mall',
       'Skate Park', 'Soup Place', 'Sporting Goods Shop', 'Steakhouse',
       'Supermarket'],
      dtype='object')

In [33]:
CBG_Grouped_Selected = CBP_Grouped[['Neighbourhoods', 
                                    'Breakfast Spot', 
                                    'Café', 
                                    'Coffee Shop', 
                                    'Discount Store', 
                                    'Food Court', 
                                    'Ice Cream Shop', 
                                    'Indian Restaurant', 
                                    'Malay Restaurant', 
                                    'Shopping Mall', 
                                    'Supermarket']]
CBG_Grouped_Selected.head()

#temp_df = CBP_Grouped.drop('Neighbourhoods', 1)
#k_clusters=3
#k_means=KMeans(n_clusters=k_clusters, random_state=0).fit(temp_df)
#k_means.labels_[0:3]

,Neighbourhoods,Breakfast Spot,Café,Coffee Shop,Discount Store,Food Court,Ice Cream Shop,Indian Restaurant,Malay Restaurant,Shopping Mall,Supermarket
0,"Bedok, Upper East Coast, Eastwood, Kew Drive",0.014085,0.070423,0.084507,0.014085,0.098592,0.028169,0.028169,0.014085,0.042254,0.014085
1,"Loyang, Changi",0.000000,0.166667,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Simei, Tampines, Pasir Ris",0.000000,0.052632,0.105263,0.000000,0.052632,0.000000,0.105263,0.000000,0.000000,0.000000


In [34]:
CBG_Grouped_Selected_Clstr = CBG_Grouped_Selected.drop('Neighbourhoods', 1)
k_clusters=2
k_means = KMeans(n_clusters=k_clusters, random_state=0).fit(CBG_Grouped_Selected_Clstr)
k_means.labels_[0:3]

array([0, 1, 0], dtype=int32)

In [35]:
CBG_Grouped_Selected.insert(1, 'Cluster_Labels', k_means.labels_)
CBG_Grouped_Selected.head()

,Neighbourhoods,Cluster_Labels,Breakfast Spot,Café,Coffee Shop,Discount Store,Food Court,Ice Cream Shop,Indian Restaurant,Malay Restaurant,Shopping Mall,Supermarket
0,"Bedok, Upper East Coast, Eastwood, Kew Drive",0,0.014085,0.070423,0.084507,0.014085,0.098592,0.028169,0.028169,0.014085,0.042254,0.014085
1,"Loyang, Changi",1,0.000000,0.166667,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Simei, Tampines, Pasir Ris",0,0.000000,0.052632,0.105263,0.000000,0.052632,0.000000,0.105263,0.000000,0.000000,0.000000


In [36]:
Neighbourhood_Clstr = CBG_Grouped_Selected[['Neighbourhoods', 'Cluster_Labels']]
Neighbourhood_Clstr.head()

,Neighbourhoods,Cluster_Labels
0,"Bedok, Upper East Coast, Eastwood, Kew Drive",0
1,"Loyang, Changi",1
2,"Simei, Tampines, Pasir Ris",0


In [37]:
CBP_Nearby_Venues_Clstr = pd.merge(CBP_Nearby_Venues, Neighbourhood_Clstr, on='Neighbourhoods')
CBP_Nearby_Venues_Clstr.head()

,name,categories,neighborhood,lat,lng,distance,address,lat_long,PostalCode,PostalSector,PostalDistrict,Neighbourhoods,Cluster_Labels
0,Huggs JPM @ Changi City (huggs coffee),Café,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1,"1.3344, 103.9642",486018.0,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive",0
1,Capri By Fraser,Hotel,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1,"1.3336, 103.9637",486038.0,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive",0
2,SONG FA Bak Kut Teh,Chinese Restaurant,NaN,1.335363,103.964323,106,UE BizHub East,"1.3354, 103.9643",486017,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive",0
3,Nike Factory Store,Sporting Goods Shop,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point","1.3344, 103.9625",486038.0,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive",0
4,Starbucks,Coffee Shop,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point,"1.3342, 103.9626",486038.0,48,16,"Bedok, Upper East Coast, Eastwood, Kew Drive",0


In [38]:
print('Clusters of venues CBP and nearby area')
Clustered_map_venue=folium.Map(location=[latitude, longitude], zoom_start=13)

x=np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(CBP_Nearby_Venues_Clstr['lat'], CBP_Nearby_Venues_Clstr['lng'], CBP_Nearby_Venues_Clstr['Neighbourhoods'], CBP_Nearby_Venues_Clstr['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Clustered_map_venue)

Clustered_map_venue

Clusters of venues CBP and nearby area


In [39]:
SG_MRT_LL_DF = pd.read_csv('SG_MRT_LL.csv')
SG_MRT_LL_DF.head()

,Sl No,Station_Name,Category,Lat,Lng
0,1,Jurong East,MRT,1.333207,103.742308
1,2,Bukit Batok,MRT,1.349069,103.749596
2,3,Bukit Gombak,MRT,1.359043,103.751863
3,4,Choa Chu Kang,MRT,1.385417,103.744316
4,5,Yew Tee,MRT,1.397383,103.747523


In [40]:
print('Map showing locations of Singapore MRT Stations')
MRT_STN_map=folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, stn_name in zip(SG_MRT_LL_DF['Lat'], SG_MRT_LL_DF['Lng'], SG_MRT_LL_DF['Station_Name']):
    label = folium.Popup('MRT Station '+str(stn_name), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        fill_color='#132b5e',
        num_sides=3,
        radius=7,
        popup=label,
        fill=True,
        fill_opacity=0.7
    ).add_to(MRT_STN_map)

MRT_STN_map

Map showing locations of Singapore MRT Stations


In [41]:
SG_Rent_DF = pd.read_csv('SG_Rent.csv')
SG_Rent_DF.head()

,Unnamed: 0,Town,3-Room,4-Room,5-Room
0,0,Ang Mo Kio,1700,2100,2400
1,1,Bedok,1700,2000,2250
2,2,Bishan,1800,2300,2500
3,3,Bukit Batok,1600,1900,2050
4,4,Bukit Merah,1950,2500,2800


In [42]:
SG_Rent_DF.drop(SG_Rent_DF.columns[0:1], 1, inplace=True)
SG_Rent_DF.head()

,Town,3-Room,4-Room,5-Room
0,Ang Mo Kio,1700,2100,2400
1,Bedok,1700,2000,2250
2,Bishan,1800,2300,2500
3,Bukit Batok,1600,1900,2050
4,Bukit Merah,1950,2500,2800


In [43]:
SG_Rent_DF.tail()

,Town,3-Room,4-Room,5-Room
22,Simei,1800,2100,2200
23,Tampines,1750,2000,2200
24,Toa Payoh,1800,2300,2600
25,Woodlands,1400,1700,1800
26,Yishun,1600,1800,2000


In [44]:
NearbyVenues_Town = NearbyVenues_PC
NearbyVenues_Town['Town'] = ''

In [46]:
Search_for_this_value = SG_Rent_DF['Town'].tolist()
for value in enumerate(Search_for_this_value):
    pattern = Search_for_this_value[value[0]]
    NearbyVenues_Town.loc[NearbyVenues_Town['address'].str.contains(pattern), 'Town'] = value[1]
NearbyVenues_Town.head()

,name,categories,neighborhood,lat,lng,distance,address,lat_long,PostalCode,PostalSector,Town
0,Huggs JPM @ Changi City (huggs coffee),Café,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1,"1.3344, 103.9642",486018.0,48,
1,Capri By Fraser,Hotel,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1,"1.3336, 103.9637",486038.0,48,Central
2,SONG FA Bak Kut Teh,Chinese Restaurant,NaN,1.335363,103.964323,106,UE BizHub East,"1.3354, 103.9643",486017,48,
3,Nike Factory Store,Sporting Goods Shop,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point","1.3344, 103.9625",486038.0,48,
4,Starbucks,Coffee Shop,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point,"1.3342, 103.9626",486038.0,48,


In [47]:
NearbyVenues_Town.loc[NearbyVenues_Town['address'].str.contains('Changi Business Park'), 'Town'] = 'Changi Business Park'
NearbyVenues_Town.head()

,name,categories,neighborhood,lat,lng,distance,address,lat_long,PostalCode,PostalSector,Town
0,Huggs JPM @ Changi City (huggs coffee),Café,NaN,1.334441,103.964183,135,8 Changi Business Park Ave 1,"1.3344, 103.9642",486018.0,48,Changi Business Park
1,Capri By Fraser,Hotel,NaN,1.333618,103.963651,238,3 Changi Business Park Central 1,"1.3336, 103.9637",486038.0,48,Changi Business Park
2,SONG FA Bak Kut Teh,Chinese Restaurant,NaN,1.335363,103.964323,106,UE BizHub East,"1.3354, 103.9643",486017,48,
3,Nike Factory Store,Sporting Goods Shop,NaN,1.334440,103.962490,312,"#02-31/32/33/34, Changi City Point","1.3344, 103.9625",486038.0,48,
4,Starbucks,Coffee Shop,Changi Business Park,1.334180,103.962616,307,#01-25 Changi City Point,"1.3342, 103.9626",486038.0,48,


In [48]:
Town_LL_DF = NearbyVenues_Town[['Town', 'lat', 'lng']]
Town_LL_DF = Town_LL_DF[Town_LL_DF['Town'] != '']
Town_LL_DF

,Town,lat,lng
0,Changi Business Park,1.334441,103.964183
1,Changi Business Park,1.333618,103.963651
6,Changi Business Park,1.335085,103.964109
10,Changi Business Park,1.335176,103.963543
15,Bedok,1.320335,103.955464
16,Bedok,1.320335,103.955464
17,Changi Business Park,1.335259,103.963645
20,Changi Business Park,1.336226,103.966541
27,Simei,1.343436,103.953487
32,Bedok,1.331267,103.947671


In [49]:
Town_LL_DF_uniq = Town_LL_DF.drop_duplicates(subset='Town', keep='last')
Town_LL_DF_uniq

,Town,lat,lng
27,Simei,1.343436,103.953487
83,Changi Business Park,1.335735,103.963485
102,Bedok,1.331714,103.947322
110,Tampines,1.344600,103.941256


#Get SG_Rent Lat, Long
for n in range(len(SG_Rent_DF)):
    address = (SG_Rent_DF['Town'][n] + ',' + ' Singapore')
    geolocator=Nominatim()
    location_r = geolocator.geocode(address)
    latitude_r = location.latitude
    longitude_r = location.longitude
    SG_Rent_DF['lat'] = latitude_r
    SG_Rent_DF['lng'] = longitude_r
    time.sleep(5)
SG_Rent_DF.to_csv('SG_Rent_LL.csv')
SG_Rent_DF.head()

address = 'Tampines'
geolocator=Nominatim()
location_r = geolocator.geocode(address)
latitude_r = location.latitude
longitude_r = location.longitude
print(latitude_r)
print(longitude_r)

In [50]:
SG_Rent_LL_DF = pd.merge(SG_Rent_DF, Town_LL_DF_uniq, on='Town', how='left')
SG_Rent_LL_DF.dropna(subset=['lat'], inplace=True)
#SG_Rent_LL_DF.drop(SG_Rent_LL_DF.columns[0:1], 1, inplace=True)
SG_Rent_LL_DF.head()

,Town,3-Room,4-Room,5-Room,lat,lng
1,Bedok,1700,2000,2250,1.331714,103.947322
22,Simei,1800,2100,2200,1.343436,103.953487
23,Tampines,1750,2000,2200,1.344600,103.941256


In [51]:
#Cluster the rates
SG_Rent_LL_Clustered = SG_Rent_LL_DF[['3-Room', '4-Room']]
SG_Rent_LL_Clustered.head()

,3-Room,4-Room
1,1700,2000
22,1800,2100
23,1750,2000


In [52]:
k_clusters_r=2
k_means_r = KMeans(n_clusters=k_clusters_r, random_state=0).fit(SG_Rent_LL_Clustered)
k_means_r.labels_[0:]

array([0, 1, 0], dtype=int32)

In [53]:
SG_Rent_LL_DF.insert(1, 'Clstr_labels', k_means_r.labels_)
SG_Rent_LL_DF

,Town,Clstr_labels,3-Room,4-Room,5-Room,lat,lng
1,Bedok,0,1700,2000,2250,1.331714,103.947322
22,Simei,1,1800,2100,2200,1.343436,103.953487
23,Tampines,0,1750,2000,2200,1.344600,103.941256


In [54]:
print('Map showing Clusters of area with similar rent conditions, nearby CBP')
Clustered_map_rent=folium.Map(location=[latitude, longitude], zoom_start=13)

x_r=np.arange(k_clusters_r)
ys_r = [i_r + x_r + (i_r*x_r)**3 for i_r in range(k_clusters_r)]
colors_array_r = cm.rainbow(np.linspace(0, 1, len(ys_r)))
rainbow_r = [colors.rgb2hex(i_r) for i_r in colors_array_r]
#print (rainbow_r)
markers_colors_r = []
for lat_r, lon_r, poi_r, cluster_r, rent_3_room, rent_4_room, rent_5_room in zip(SG_Rent_LL_DF['lat'], SG_Rent_LL_DF['lng'], SG_Rent_LL_DF['Town'], SG_Rent_LL_DF['Clstr_labels'], SG_Rent_LL_DF['3-Room'], SG_Rent_LL_DF['4-Room'], SG_Rent_LL_DF['5-Room']):
    popup_value = 'Town : ' + str(poi_r) + '; Rent - 3 Room : ' + str(rent_3_room) + '; 4 Room : ' + str(rent_4_room) + '; 5 Room : ' + str(rent_5_room)
    label = folium.Popup(popup_value, parse_html=True)
    folium.CircleMarker(
        [lat_r, lon_r],
        radius=30,
        popup=label,
        color=rainbow_r[cluster_r-1],
        fill=True,
        fill_color=rainbow_r[cluster_r-1],
        fill_opacity=0).add_to(Clustered_map_rent)

Clustered_map_rent

Map showing Clusters of area with similar rent conditions, nearby CBP


In [55]:
print("Consolidated Map for Venues, MRT and Rental Rates clusters")
Clustered_map=folium.Map(location=[latitude, longitude], zoom_start=13)

x=np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(CBP_Nearby_Venues_Clstr['lat'], CBP_Nearby_Venues_Clstr['lng'], CBP_Nearby_Venues_Clstr['Neighbourhoods'], CBP_Nearby_Venues_Clstr['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Clustered_map)
    
for lat, lng, stn_name in zip(SG_MRT_LL_DF['Lat'], SG_MRT_LL_DF['Lng'], SG_MRT_LL_DF['Station_Name']):
    label = folium.Popup('MRT Station '+str(stn_name), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        fill_color='#132b5e',
        num_sides=3,
        radius=7,
        popup=label,
        fill=True,
        fill_opacity=0.7
    ).add_to(Clustered_map)

    
x_r=np.arange(k_clusters_r)
ys_r = [i_r + x_r + (i_r*x_r)**3 for i_r in range(k_clusters_r)]
colors_array_r = cm.rainbow(np.linspace(0, 1, len(ys_r)))
rainbow_r = [colors.rgb2hex(i_r) for i_r in colors_array_r]
#print (rainbow_r)
markers_colors_r = []
for lat_r, lon_r, poi_r, cluster_r, rent_3_room, rent_4_room, rent_5_room in zip(SG_Rent_LL_DF['lat'], SG_Rent_LL_DF['lng'], SG_Rent_LL_DF['Town'], SG_Rent_LL_DF['Clstr_labels'], SG_Rent_LL_DF['3-Room'], SG_Rent_LL_DF['4-Room'], SG_Rent_LL_DF['5-Room']):
    popup_value = 'Town : ' + str(poi_r) + '; Rent - 3 Room : ' + str(rent_3_room) + '; 4 Room : ' + str(rent_4_room) + '; 5 Room : ' + str(rent_5_room)
    label = folium.Popup(popup_value, parse_html=True)
    folium.CircleMarker(
        [lat_r, lon_r],
        radius=30,
        popup=label,
        color=rainbow_r[cluster_r-1],
        fill=True,
        fill_color=rainbow_r[cluster_r-1],
        fill_opacity=0).add_to(Clustered_map)

Clustered_map

Consolidated Map for Venues, MRT and Rental Rates clusters


Result Section :

From the above map we observe the below:
    1. There are two clusters for venues within 3 KM from Changi Business Park.
    2. Map shows the Locations of MRT Stations.
    3. There are two clusters of similar Rentable places.
    4. On exploring venues it is observed that Venues marked as Red (Spread across, CBP, Simei, Bedok, Tampinies and Pasir Ris) are more suitable.
    5. On exploring the MRT Plot, most suitable places are Bedok, Tampinies and Simei
    6. From the clusters o rent it can be concluded that either Bedok or Tampines are most suitable place to look for the house, though Simei is also not very costly.

4. Result Section :

From the above map we observe the below:
    
    1. There are two clusters for venues within 3 KM from Changi Business Park.
    2. Map shows the Locations of MRT Stations.
    3. There are two clusters of similar Rentable places.
    4. On exploring venues it is observed that Venues marked as Red (Spread across, CBP, Simei, Bedok, Tampinies and Pasir Ris) are more suitable.
    5. On exploring the MRT Plot, most suitable places are Bedok, Tampinies and Simei
    6. From the clusters of rent it can be concluded that either Bedok or Tampines are most suitable place to look for the house, though Simei is also not very costly.
   

5. Discussion Section :

From the above results It can be concluded that:
    
    1. Bedok, Simei, Tampinese and Loyang Changi are the nearby places to Changi Business Park.
    2. Whereas Loyang Changi is more suitable area for tourists and visitor for availability of mostly resort and other similar places, Bedok, Tampines and Simei are more suitable for resindential purposes.
    3. Both Bedok and Tampines area meets the selection prefrences.
    4. CBP Neighbourhood has good connectivity to nearby places through MRT and all the three point of interest have nearby MRT stations.
    5. There is not much difference of Rent Rates in various residential areas nearby to CBP, however Bedok and Tampines are cheaper when compared to Simei.
    6. Bedok is a big area and only Partially covered under 3 KM range
From the above discussion it can be concluded that the most suitable place for stay of Indian near CBP is Tampines.   

6. Conclusion Section :

From above discussion it is concluded that Tampines is the most suitable location for a Person who is moving in to CBP from India.